# Data Wrangling: Clean, Transform, Merge, Reshape
## Combining and Merging Data Sets
- pandas.merge : connects rows in DataFrames based on one or more keys (SQL *join*)
- pandas.concat : glues or stacks together objects along an axis
- combine_first : enables slicing together overlapping data to fill in missing values in one object with values from values

### Database-style DataFrame Merges
*merge* or *join* : combine data sets by linking rows using one or more *keys*

In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

In [ ]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                'data1': range(7)})
df1

In [ ]:
df2 = DataFrame({'key': ['a', 'b', 'd'],
                'data2': range(3)})
df2

In [ ]:
pd.merge(df1, df2)

In [ ]:
pd.merge(df1, df2, on='key')

In [ ]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})

In [ ]:
df3

In [ ]:
df4

In [ ]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

In [ ]:
# how = 'outer'
pd.merge(df1, df2, how='outer')  # default : inner

In [ ]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                'data2': range(5)})

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, how='left')

In [ ]:
# To merge with multiple keys
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

In [ ]:
# overlapping column names
pd.merge(left, right, on='key1')

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

*merge function arguments*

|Argument|Description|
|---------|--------------|
|left||
|right||
|how|**'inner'**, 'outer', 'left', 'right'|
|on||
|left_on||
|right_on||
|left_index||
|right_index||
|sort||
|suffixes||
|copy||

### Merging on Index

In [ ]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a','b'])

In [ ]:
left1

In [ ]:
right1

In [ ]:
# left_index=True, right_index=True
pd.merge(left1, right1, left_on='key', right_index=True)

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

In [ ]:
# Hierachically-indexed data
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                  'key2': [2000, 2001, 2002, 2001, 2002],
                  'data': np.arange(5.)})

righth = DataFrame(np.arange(12).reshape((6, 2)),
                  index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                         [2001, 2000, 2000, 2000, 2001, 2002]],
                  columns=['envent1', 'event2'])

In [ ]:
lefth

In [ ]:
righth

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

In [ ]:
# Using the indexes of both sides
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                  columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

In [ ]:
left2

In [ ]:
right2

In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

In [ ]:
# join() : for merging by index
left2.join(right2, how='outer')

In [ ]:
# also supports joining the index of the passed DataFrame on one of the columns of the calling DataFrame
left1.join(right1, on='key')

In [ ]:
# for simple index-on-index merges
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])

In [ ]:
left2.join([right2, another])

In [ ]:
left2.join([right2, another], how='outer')

### Concatenating Along an Axis

In [ ]:
# Numpy : concatenate()
arr = np.arange(12).reshape((3,4))
arr

In [ ]:
np.concatenate([arr, arr], axis=1)

In [ ]:
s1 = Series([0, 1], index=['a', 'b'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])

In [ ]:
pd.concat([s1, s2, s3])

In [ ]:
# if axis = 1,
pd.concat([s1, s2, s3], axis=1)

*concat function argumennts*

|Argument|Description|
|--------|------------|
|objs||
|axis||
|join||
|join_axes||
|keys||
|levels||
|names||
|verify_integrity||
|ignore_index||

### Combining Data with Overlap

In [ ]:
# Numpy's where
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [ ]:
a

In [ ]:
b

In [ ]:
np.where(pd.isnull(a), b, a)

In [ ]:
# combine_first() : "patching" missing data

In [ ]:
b[:-2].combine_first(a[2:])

In [ ]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})

df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1.combine_first(df2)

## Reshaping and Pivoting
### Reshaping with Hierarchical Indexing
- **stack**: this "rotates" or pivots from the columns in the data to the rows
- **unstack**: this pivots from the rows into the columns

In [ ]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

In [ ]:
# stack()
result = data.stack()
result

In [ ]:
# unstack()
result.unstack()

In [ ]:
# unstack a diffrerent level - by default the innermost level is unstacked
result.unstack(0)

In [ ]:
result.unstack('state')

In [ ]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2.unstack()

In [ ]:
data2.unstack().stack()

In [ ]:
data2.unstack().stack(dropna=False)

In [ ]:
# When unstacking in a DataFrame, the level unstacked becomes the lowest level in the result
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

In [ ]:
df.unstack('state')

In [ ]:
df.unstack('state').stack('side')

## Data Transformation
### Removing Duplicates

In [ ]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
# duplicated()
data.duplicated()

In [ ]:
# drop_duplicated()
data.drop_duplicates()

In [ ]:
# filter duplicates only based on the 'k1' columns
data['v1'] = range(7)
data.drop_duplicates(['k1'])

In [ ]:
# keep='last' (python 2.x : take_last=True)
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Functional or Mapping

In [ ]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

data

In [ ]:
# add a columnb indicating the type of animal that each food came from
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmom'
}

In [ ]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
                                                 
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### Replacing Values

In [ ]:
data = Series([1., -999., 2., -999., -1000., 3.])

data

In [ ]:
# replace -999 to NA
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes

In [ ]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data.index.map(str.upper)

In [ ]:
data.index = data.index.map(str.upper)

In [ ]:
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
# inplace=True
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [ ]:
data

### Discretization and Binning

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

In [ ]:
cats = pd.cut(ages, bins)

cats

In [ ]:
cats.codes

In [ ]:
cats.categories

In [ ]:
pd.value_counts(cats)

In [ ]:
# () : open, [] : closed
# right=False
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAges', 'Senior']

In [ ]:
pd.cut(ages, bins, labels=group_names)

In [ ]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

In [ ]:
# qcut : bins the data based on sample quantiles
data = np.random.randn(1000)
cats = pd.qcut(data, 4)

cats

In [ ]:
pd.value_counts(cats)

In [ ]:
# your own quantiles(numbers between 0 and 1)
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

### Detecting and Filtering Outliers

In [ ]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))

In [ ]:
data.describe()

In [ ]:
col = data[3]
col[np.abs(col) > 3]

In [ ]:
# to select all rows having a value exceeding 3 or -3
data[(np.abs(data) > 3).any(1)]    # 1 = 'columns'

In [ ]:
# cap values outside the interval -3 to 3
data[np.abs(data) > 3] = np.sign(data) * 3    # sign() : -1 if x < 0, 0 if x==0, 1 if x > 0

In [ ]:
data.describe()

### Permutation and Random Sampling

In [ ]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))

In [ ]:
sampler = np.random.permutation(5)

In [ ]:
sampler

In [ ]:
df

In [ ]:
df.take(sampler)

In [ ]:
#  random sampling with permutation
df.take(np.random.permutation(len(df))[:3])

In [ ]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)

In [ ]:
sampler

In [ ]:
draws = bag.take(sampler)

draws

### Computing Indicator/Dummy Variables 

In [ ]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1':range(6)})

In [ ]:
pd.get_dummies(df['key'])

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [ ]:
df_with_dummy = df[['data1']].join(dummies)

In [ ]:
df_with_dummy

In [ ]:
values = np.random.randn(10)
values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

## String Manipulation
### String Object Methods

In [ ]:
val = 'a, b, guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',')]  # 'strip' - trim whitespace(include newlines)

In [ ]:
pieces

In [ ]:
first, second, third = pieces

In [ ]:
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val

In [ ]:
val.index(',')

In [ ]:
val.find(':')
## index : exception, find : return -1

In [ ]:
# count
val.count(',')

In [ ]:
# replace
val.replace(',', '::')

In [ ]:
val.replace(',', '')

*Python built-in string methods*

|Argument|Description|
|--------|---------------|
|count||
|endswith, startswith||
|join||
|index||
|find||
|rfind||
|replace||
|strip, rstrip, lstrip||
|split||
|lower, upper||
|ljust, rjust||

------

↓↓

### Regular expressions

In [ ]:
import re

In [ ]:
text = "foo      bar\t baz  \tqux"
re.split('\s+', text)

In [ ]:
# reusable regex object
regex = re.compile('\s+')

regex.split(text)

In [ ]:
regex.findall(text)

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
regex.findall(text)

In [ ]:
m = regex.search(text)  # search() returns a special match object for the first emaill address in the text

In [ ]:
m

In [ ]:
text[m.start():m.end()]

In [ ]:
print(regex.match(text))

In [ ]:
print(regex.sub('REDACTED', text))

In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags=re.IGNORECASE)

In [ ]:
m = regex.match('wesm@bright.net')

m.groups()

In [ ]:
regex.findall(text)

In [ ]:
# \1, \2 ...
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

In [ ]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [ ]:
m = regex.match('wesm@bright.net')

m.groupdict()

*Regular expression methodS*

|Argument|Description|
|---------|------------|
|findall, finditer||
|match||
|search||
|split||
|sub, subn||

### Vectorized string functions in pandas

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [ ]:
data

In [ ]:
data.str.contains('gamil')

In [ ]:
pattern

In [ ]:
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

In [ ]:
matches.str.get(1)

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]

*Vectorized string methods*

|Method|Description|
|------|--------------|
|cat||
|contains||
|count||
|endswith, startswith||
|findall||
|get||
|join||
|len||
|lower, upper||
|match||
|pad||
|center||
|repeat||
|replace||
|slice||
|split||
|strip, rstrip, lstrip||